In [1]:
from kplib.dumbo import Dumbo, FeatureOptions, EvaluationOptions, DataframeInput, FileInput
import pandas as pd
import numpy as np
from pysnowflake.simple import Session
import blocks

/usr/local/lib/python3.7/dist-packages/pysnowflake/__init__.py:21: FutureWarning: 
    With pysnowflake version 1.0.0 there will be a breaking change.
    The legacy api will be dropped as it is no longer maintained, and simple will become the only api.
    This will make it easier to keep support for newer features like AWS support, etc. The expected change is
    the start of Q2 2022.
    
  FutureWarning,


In [2]:
def PullToken(st_dt, ed_dt):
    with Session() as ss:
        df  = ss.download(query="""
            SELECT GET(entity_tokens, 'unit_token') as unit_token
            , decision_event_token as decision_event_token
            , created_at as created_at
            , status
            FROM decisions.public.decisions_data_api dda
            -- Filter by Date
            WHERE created_at >= '{}' ::DATE
            AND created_at < '{}' ::DATE
            --   Get Production decisions only
            AND is_shadow=FALSE
            AND decision_type = 'ELIGIBILITY';
            """.format(st_dt, ed_dt)) 
    return df

### 1.1 April

In [3]:
df4 = PullToken('2022-04-01', '2022-05-01')

In [4]:
df4.shape

(46764810, 4)

In [5]:
df4.head()

,unit_token,decision_event_token,created_at,status
0,4VA6FRXFBXVAG,DS-78e837f7-8acc-4641-9be2-ec22a773de6d,2022-04-27 04:02:38.100,declined
1,TPG31G92QSJ6K,DS-c07c4e83-f66a-4411-8352-7d556406ce45,2022-04-27 03:47:13.076,approved
2,9NSP9YMA3ES73,DS-0b14fff1-e13c-4962-9c06-644e9cbfe1af,2022-04-27 03:34:38.107,approved
3,0ESP2RANPNSAK,DS-f74d1820-fff5-4abb-9831-f063ef78d961,2022-04-27 04:01:33.098,declined
4,D2Z1C8HBKZQ2S,DS-7997c03a-3035-4c84-9742-f7b9f146052c,2022-04-27 04:08:38.106,declined


In [7]:
url4 = "gs://sq-ds-capital-prod/mrm/Eligibility_2022/TokenList_eligibility_202204.pq"
blocks.place(df4, url4)

In [6]:
df4['date'] = df4['created_at'].dt.date.apply(lambda x: x.strftime('%Y-%m-%d'))

In [7]:
df4

,unit_token,decision_event_token,created_at,status,date
0,4VA6FRXFBXVAG,DS-78e837f7-8acc-4641-9be2-ec22a773de6d,2022-04-27 04:02:38.100,declined,2022-04-27
1,TPG31G92QSJ6K,DS-c07c4e83-f66a-4411-8352-7d556406ce45,2022-04-27 03:47:13.076,approved,2022-04-27
2,9NSP9YMA3ES73,DS-0b14fff1-e13c-4962-9c06-644e9cbfe1af,2022-04-27 03:34:38.107,approved,2022-04-27
3,0ESP2RANPNSAK,DS-f74d1820-fff5-4abb-9831-f063ef78d961,2022-04-27 04:01:33.098,declined,2022-04-27
4,D2Z1C8HBKZQ2S,DS-7997c03a-3035-4c84-9742-f7b9f146052c,2022-04-27 04:08:38.106,declined,2022-04-27
...,...,...,...,...,...
46764805,CVAK0M4TB83Y6,DS-cca7368c-b0bd-4590-af6a-7bce605ed4a7,2022-04-25 11:40:04.111,declined,2022-04-25
46764806,AJN19JNC0P4WD,DS-cf9bd54f-40f0-43fb-b45b-0ec309578d76,2022-04-25 11:35:14.084,declined,2022-04-25
46764807,WE7J09TWBMP3H,DS-8fecc1fe-aade-4489-9953-5bd4935fd755,2022-04-25 12:11:10.088,declined,2022-04-25
46764808,8QMHJ0R83Z8SR,DS-b72b6507-69bd-4ad1-8e93-143c5796665c,2022-04-25 12:01:19.234,declined,2022-04-25


In [8]:
df4['unit_token'].groupby(df4.date).count()

date
2022-04-01    1581082
2022-04-02    1636877
2022-04-03    1567368
2022-04-04    1553032
2022-04-05    1539412
2022-04-06    1523861
2022-04-07    1544079
2022-04-08    1582883
2022-04-09    1641407
2022-04-10    1546937
2022-04-11    1576615
2022-04-12    1548298
2022-04-13    1525794
2022-04-14    1544516
2022-04-15    1574819
2022-04-16    1593468
2022-04-17    1485347
2022-04-18    1455323
2022-04-19    1474400
2022-04-20    1476823
2022-04-21    1535893
2022-04-22    1580125
2022-04-23    1639689
2022-04-24    1567541
2022-04-25    1543325
2022-04-26    1547639
2022-04-27    1539172
2022-04-28    1555319
2022-04-29    1605656
2022-04-30    1678110
Name: unit_token, dtype: int64

In [38]:
df4_sample = df4[['unit_token', 'decision_event_token', 'date']].groupby('date', group_keys=False).sample(n = 100000, random_state = 2022).reset_index(drop=True)

In [40]:
df4_sample

,unit_token,decision_event_token,date
0,15YMZHEQ03TDK,DS-b524c4a9-b31a-456e-81ab-3b3c230ec3cc,2022-04-01
1,3F1YCTC04TMMC,DS-c9ebe8bc-69b2-4ae1-82f1-9a6bb4a69f26,2022-04-01
2,LQHVDABGXJFKZ,DS-86e4ee3c-a064-4448-8584-92d5ac7a1898,2022-04-01
3,BXRJXP76TKVER,DS-5aa14802-0895-4123-a034-e2c138fbe076,2022-04-01
4,WGSCBJ6BRB2P3,DS-3187e88b-1fab-4ec0-8e0e-4c5c48678318,2022-04-01
...,...,...,...
2999995,61Z3A5P4WVZEZ,DS-142d744e-1068-412f-af8f-2745cec2199d,2022-04-30
2999996,3S6WRX68XA79S,DS-9e273fec-d39f-48fd-95d9-d329745acbe0,2022-04-30
2999997,L24RX3ESAK64J,DS-53a42aa6-6ebc-4a1b-a35a-120ac4204623,2022-04-30
2999998,LHFQGTA150S6G,DS-9ae290a6-6d26-4393-83f7-132fe30d3ed2,2022-04-30


In [41]:
url4s = "gs://sq-ds-capital-prod/mrm/Eligibility_2022/TokenSample_202204.pq"
blocks.place(df4_sample, url4s)

### 1.2 May

In [14]:
df5 = PullToken('2022-05-01', '2022-06-01')

In [9]:
df5.shape

(50512757, 4)

In [10]:
df5.head()

,unit_token,decision_event_token,created_at,status
0,XRMTERDRRP8H2,DS-8b9a4625-b29a-4a79-abc5-2ef5cb17cbb3,2022-05-01 04:43:42.108,approved
1,NP76P1MAABMWY,DS-f2b5bd79-5b0a-4cd9-a9f3-549b491a228b,2022-05-01 04:54:06.100,approved
2,7SRDD4YV0K738,DS-dd96aca3-78b2-488d-9379-1fe642739524,2022-05-01 04:45:36.081,declined
3,9RHW6VCSY6435,DS-07f9f3a9-5b5d-42bc-9a4e-b9fa906a9523,2022-05-01 04:29:26.077,approved
4,5DRTWR73AF294,DS-613278a6-e1d8-4dcd-a01c-7c673d3023c1,2022-05-01 04:38:11.111,declined


In [11]:
url5 = "gs://sq-ds-capital-prod/mrm/Eligibility_2022/TokenList_eligibility_202205.pq"
blocks.place(df5, url5)

In [42]:
df5['date'] = df5['created_at'].dt.date.apply(lambda x: x.strftime('%Y-%m-%d'))

In [43]:
df5['unit_token'].groupby(df5.date).count()

date
2022-05-01    1624727
2022-05-02    1609096
2022-05-03    1612637
2022-05-04    1588604
2022-05-05    1605290
2022-05-06    1688539
2022-05-07    1763257
2022-05-08    1656617
2022-05-09    1632169
2022-05-10    1617577
2022-05-11    1585735
2022-05-12    1621594
2022-05-13    1659503
2022-05-14    1720662
2022-05-15    1646648
2022-05-16    1627669
2022-05-17    1624651
2022-05-18    1602225
2022-05-19    1622308
2022-05-20    1661174
2022-05-21    1719219
2022-05-22    1635861
2022-05-23    1605875
2022-05-24    1601821
2022-05-25    1586546
2022-05-26    1607842
2022-05-27    1650402
2022-05-28    1693431
2022-05-29    1604381
2022-05-30    1503275
2022-05-31    1533422
Name: unit_token, dtype: int64

In [44]:
df5_sample = df5[['unit_token', 'decision_event_token', 'date']].groupby('date', group_keys=False).sample(n = 100000, random_state = 2022).reset_index(drop=True)

In [45]:
df5_sample

,unit_token,decision_event_token,date
0,CYNF1VAGE6BSQ,DS-fd0ae020-6484-4bb1-90a8-7a30ebb736f5,2022-05-01
1,606H6EXHMTGGY,DS-e82d958a-0019-4409-a6a9-4b92cf184a4c,2022-05-01
2,9Q9FJ21661RN8,DS-ed9616a6-5b53-476d-a880-ee4525cde129,2022-05-01
3,L3555MV8GEZN6,DS-e23ba180-e3e2-4644-8a7c-29e02245a4c9,2022-05-01
4,B1NDPFHFW8S2D,DS-39125b59-8219-4662-9b21-e7d9dc433ed5,2022-05-01
...,...,...,...
3099995,QER6JGAW8A30F,DS-6980e894-a0cc-480a-867f-04f0b4358f94,2022-05-31
3099996,A52208PREZD3X,DS-45f29d11-c1eb-4a55-b7de-5fad708eabc6,2022-05-31
3099997,L9MQATYJ5EMCT,DS-a768a41d-97ff-48c8-8278-e07c686eaac7,2022-05-31
3099998,LH1QJVC45SZ8F,DS-48d9b5d3-fd00-4494-aa4a-52fefeee29c6,2022-05-31


In [46]:
url5s = "gs://sq-ds-capital-prod/mrm/Eligibility_2022/TokenSample_202205.pq"
blocks.place(df5_sample, url5s)

### 1.3 June

In [3]:
df6 = PullToken('2022-06-01', '2022-07-01')

In [4]:
df6.shape

(75100946, 4)

In [5]:
df6.head()

,unit_token,decision_event_token,created_at,status
0,LRZ6BWM4Q467Y,DS-CS_C-1V1D1NZ3C0YX1K_2022-06-02,2022-06-02 23:06:40.830,declined
1,F1GJGTMG6K3J9,DS-CS_C-1VRQAPNDK6PMPN_2022-06-02,2022-06-02 23:04:53.382,declined
2,LX2EGYBF6GKHB,DS-CS_C-AAEBJE0RNFNR2C_2022-06-02,2022-06-02 23:23:55.147,declined
3,LQXR7QK71FWR9,DS-CS_C-3DYDJYG5WBNH86_2022-06-02,2022-06-02 22:41:10.450,declined
4,LG7BSDWWN683X,DS-CS_C-2MD5140A7BXNKA_2022-06-02,2022-06-02 22:39:56.752,declined


In [6]:
url6 = "gs://sq-ds-capital-prod/mrm/Eligibility_2022/TokenList_eligibility_202206.pq"
blocks.place(df6, url6)

In [4]:
df6['date'] = df6['created_at'].dt.date.apply(lambda x: x.strftime('%Y-%m-%d'))

In [5]:
df6['unit_token'].groupby(df6.date).count()

date
2022-06-01    1572609
2022-06-02    2662771
2022-06-03    3811149
2022-06-04    3640376
2022-06-05    3590687
2022-06-06    3292021
2022-06-07    3310330
2022-06-08    2730126
2022-06-09    1630220
2022-06-10    1672267
2022-06-11    1731952
2022-06-12    1648972
2022-06-13    1622134
2022-06-14    1619500
2022-06-15    1609751
2022-06-16    1633196
2022-06-17    1676250
2022-06-18    1739724
2022-06-19    1640936
2022-06-20    1608913
2022-06-21    1603309
2022-06-22    3149764
2022-06-23    3234305
2022-06-24    3257658
2022-06-25    3363257
2022-06-26    3329876
2022-06-27    3262615
2022-06-28    3241729
2022-06-29    3227717
2022-06-30    2986832
Name: unit_token, dtype: int64

In [6]:
df6_sample = df6[['unit_token', 'decision_event_token', 'date']].groupby('date', group_keys=False).sample(n = 100000, random_state = 2022).reset_index(drop=True)

In [7]:
df6_sample

,unit_token,decision_event_token,date
0,Z87JTCG5NQESV,DS-94ce4614-7ab5-4843-8de7-e5dfc8045bc5,2022-06-01
1,5YR30GAHCV27P,DS-4bfe73bd-8cd0-4833-8acc-39ba73236565,2022-06-01
2,LE3W7Y889K5B8,DS-1dd52b0d-f82f-4f58-aab7-814c1b062ee2,2022-06-01
3,8CFDM0HRCY83V,DS-655bdfdb-a910-4f27-a738-ac40158960c6,2022-06-01
4,LHCCQP21E9Q2H,DS-9f311a71-1f71-4a81-af48-0048a1b61ee2,2022-06-01
...,...,...,...
2999995,05J2T7S8RFZKR,DS-2e0df85f-5c15-47e0-b34b-f89d0e7a45cd,2022-06-30
2999996,BRXFG92MYPHPC,DS-CS_C-13GF22BJR3PS3V_2022-06-30,2022-06-30
2999997,LDJC6VXG0ZR0N,DS-9384be15-c2eb-4d54-b072-58eb255aaf7a,2022-06-30
2999998,2ED3D8XSMHKD1,DS-ddbf471e-fe9e-4762-a272-094675a3ade0,2022-06-30


In [8]:
url6s = "gs://sq-ds-capital-prod/mrm/Eligibility_2022/TokenSample_202206.pq"
blocks.place(df6_sample, url6s)

### 1.4 July

In [9]:
df7 = PullToken('2022-07-01', '2022-08-01')

In [8]:
df7.shape

(59277320, 4)

In [9]:
df7.head()

,unit_token,decision_event_token,created_at,status
0,GVYGTYQ52TSRY,DS-CS_C-E39X1R03XSTRJ2_2022-07-26,2022-07-26 19:26:27.571,declined
1,ZZ938TBHTFV93,DS-CS_C-1H0AYTCBC87AD5_2022-07-26,2022-07-26 19:38:58.707,declined
2,LX2F2498ABCD6,DS-CS_C-39ERN7F72C87SQ_2022-07-26,2022-07-26 19:29:28.191,declined
3,CH2R6SWDN4YHT,DS-CS_C-NB2RABHCGH90P2_2022-07-26,2022-07-26 20:18:27.101,declined
4,DC5E7NEKWZX1R,DS-CS_C-5G9H98SGEA0KCH_2022-07-26,2022-07-26 20:00:15.068,declined


In [10]:
url7 = "gs://sq-ds-capital-prod/mrm/Eligibility_2022/TokenList_eligibility_202207.pq"
blocks.place(df7, url7)

In [10]:
df7['date'] = df7['created_at'].dt.date.apply(lambda x: x.strftime('%Y-%m-%d'))

In [11]:
df7['unit_token'].groupby(df7.date).count()

date
2022-07-01    3360019
2022-07-02    3342231
2022-07-03    3349527
2022-07-04    3280774
2022-07-05    3033380
2022-07-06    2519550
2022-07-07    1518154
2022-07-08    1564861
2022-07-09    1639734
2022-07-10    1674137
2022-07-11    1628580
2022-07-12    1608867
2022-07-13    1602474
2022-07-14    1581325
2022-07-15    1545310
2022-07-16    1589513
2022-07-17    1791408
2022-07-18    1639420
2022-07-19    1464361
2022-07-20    1643559
2022-07-21    1635358
2022-07-22    1669046
2022-07-23    1638496
2022-07-24    1673399
2022-07-25    1622906
2022-07-26    1597889
2022-07-27    1588369
2022-07-28    1568719
2022-07-29    1593699
2022-07-30    1636326
2022-07-31    1675929
Name: unit_token, dtype: int64

In [12]:
df7_sample = df7[['unit_token', 'decision_event_token', 'date']].groupby('date', group_keys=False).sample(n = 100000, random_state = 2022).reset_index(drop=True)

In [13]:
df7_sample

,unit_token,decision_event_token,date
0,LWY8ACWX93ANP,DS-CS_C-2HKC3BWVNR37VE_2022-07-01,2022-07-01
1,KQTTMC4PER3M4,DS-b6ee2b32-f184-4696-9de7-f73ba63e7f05,2022-07-01
2,LFYDZD9TC2XB6,DS-03f014fd-da8b-42c7-bfae-15f5f45e92d7,2022-07-01
3,GE1ND8EXP6RG5,DS-CS_C-2041REHCD3J5ES_2022-07-01,2022-07-01
4,25JFBSS3SYKDP,DS-CS_C-3GDQBHNM85MFV8_2022-07-01,2022-07-01
...,...,...,...
3099995,5KB4CQCGB0SBE,DS-CS_C-29C8R2F7T3HN8P_2022-07-31,2022-07-31
3099996,QREJ86TT9CHS3,DS-CS_C-1XNY7QJJW0GB8W_2022-07-31,2022-07-31
3099997,BDHJ1HHBGR11X,DS-CS_C-JAQS9906HK0ESK_2022-07-31,2022-07-31
3099998,LT1CRFN6K6HFR,DS-CS_C-2CSNYFN2A3RQZB_2022-07-31,2022-07-31


In [14]:
url7s = "gs://sq-ds-capital-prod/mrm/Eligibility_2022/TokenSample_202207.pq"
blocks.place(df7_sample, url7s)

### 1.5 August

In [15]:
df8 = PullToken('2022-08-01', '2022-09-01')

In [12]:
df8.shape

(50949611, 4)

In [13]:
df8.head()

,unit_token,decision_event_token,created_at,status
0,LPB9JWY18KF80,DS-CS_C-PC56G9K12RH9JS_2022-08-03,2022-08-03 19:06:11.106,declined
1,LDJSRDFBHMHAK,DS-CS_C-1V9THW0MF2K910_2022-08-03,2022-08-03 18:55:45.586,declined
2,L5JR770GFB9F5,DS-CS_C-3RBDRSPBHTQ76Y_2022-08-03,2022-08-03 18:58:52.561,declined
3,7629AHT1T841H,DS-CS_C-3FQ2J7TYHFCP37_2022-08-03,2022-08-03 19:08:56.307,declined
4,ENRC334GSVGPK,DS-CS_C-1G6N3A11MDY60V_2022-08-03,2022-08-03 19:05:01.205,declined


In [14]:
url8 = "gs://sq-ds-capital-prod/mrm/Eligibility_2022/TokenList_eligibility_202208.pq"
blocks.place(df8, url8)

In [16]:
df8['date'] = df8['created_at'].dt.date.apply(lambda x: x.strftime('%Y-%m-%d'))

In [17]:
df8['unit_token'].groupby(df8.date).count()

date
2022-08-01    1631740
2022-08-02    1621323
2022-08-03    1621343
2022-08-04    1607129
2022-08-05    1628465
2022-08-06    1666723
2022-08-07    1705579
2022-08-08    1657750
2022-08-09    1635170
2022-08-10    1627227
2022-08-11    1602936
2022-08-12    1617664
2022-08-13    1654495
2022-08-14    1702817
2022-08-15    1658324
2022-08-16    1640478
2022-08-17    1637911
2022-08-18    1609785
2022-08-19    1621772
2022-08-20    1660019
2022-08-21    1707666
2022-08-22    1662267
2022-08-23    1637890
2022-08-24    1627883
2022-08-25    1597087
2022-08-26    1609556
2022-08-27    1652457
2022-08-28    1705291
2022-08-29    1665647
2022-08-30    1641022
2022-08-31    1634195
Name: unit_token, dtype: int64

In [18]:
df8_sample = df8[['unit_token', 'decision_event_token', 'date']].groupby('date', group_keys=False).sample(n = 100000, random_state = 2022).reset_index(drop=True)

In [19]:
df8_sample

,unit_token,decision_event_token,date
0,LC494X9J7NE7K,DS-CS_C-2G7KF8W826VJT3_2022-08-01,2022-08-01
1,FPTQH8XKT38CK,DS-CS_C-326FARMQ84HE4E_2022-08-01,2022-08-01
2,0ZT9D6M2PS0E4,DS-CS_C-9ZJP9EMB89ST15_2022-08-01,2022-08-01
3,5CZGJFJFPNGE4,DS-CS_C-3PPDQXCC3VMJHT_2022-08-01,2022-08-01
4,LSKZTPPNAK67W,DS-CS_C-QMSBPEKVJMRKE8_2022-08-01,2022-08-01
...,...,...,...
3099995,3FTEME3D7RB6M,DS-CS_C-2945TX285Z7FXA_2022-08-31,2022-08-31
3099996,LFHQE9X536FS9,DS-CS_C-A1VDWBA4J5MCZ5_2022-08-31,2022-08-31
3099997,2XEYD5QN6VRYT,DS-CS_C-1NNM1WC124E3AF_2022-08-31,2022-08-31
3099998,F2J1AC8NPGCE1,DS-CS_C-NCH1SS7FNB7A12_2022-08-31,2022-08-31


In [20]:
url8s = "gs://sq-ds-capital-prod/mrm/Eligibility_2022/TokenSample_202208.pq"
blocks.place(df8_sample, url8s)

### 1.6 September

In [3]:
df9 = PullToken('2022-09-01', '2022-10-01')

In [16]:
df9.shape

(50029146, 4)

In [17]:
df9.head()

,unit_token,decision_event_token,created_at,status
0,L1ZZF5GPNGHGT,DS-CS_C-35HTTGR7ACNSDH_2022-09-02,2022-09-02 15:03:35.380,approved
1,X7WXZ8DK3GR98,DS-CS_C-ZRV9ST3VQRQ204_2022-09-02,2022-09-02 15:03:39.217,approved
2,99K94F7QJCTJM,DS-CS_C-17WNNQ1PAX8HFF_2022-09-02,2022-09-02 15:06:11.203,approved
3,0XEQE92X8EJ1H,DS-CS_C-2BG9QMP3SAEQP9_2022-09-02,2022-09-02 15:04:01.122,declined
4,LGS676NQ7XM9D,DS-CS_C-ZKP4X839E60BE6_2022-09-02,2022-09-02 15:01:27.703,approved


In [18]:
url9 = "gs://sq-ds-capital-prod/mrm/Eligibility_2022/TokenList_eligibility_202209.pq"
blocks.place(df9, url9)

In [4]:
df9['date'] = df9['created_at'].dt.date.apply(lambda x: x.strftime('%Y-%m-%d'))

In [5]:
df9['unit_token'].groupby(df9.date).count()

date
2022-09-01    1613092
2022-09-02    1639518
2022-09-03    1692371
2022-09-04    1735412
2022-09-05    1681327
2022-09-06    1001121
2022-09-07    2150692
2022-09-08    1569478
2022-09-09    1607560
2022-09-10    1678509
2022-09-11    1748214
2022-09-12    1704270
2022-09-13    1679991
2022-09-14    1670876
2022-09-15    1627992
2022-09-16    1639947
2022-09-17    1691109
2022-09-18    1767110
2022-09-19    1728355
2022-09-20    1695292
2022-09-21    1683921
2022-09-22    1638479
2022-09-23    1632394
2022-09-24    1680879
2022-09-25    1753727
2022-09-26    1711530
2022-09-27    1685752
2022-09-28    1677802
2022-09-29    1625896
2022-09-30    1616530
Name: unit_token, dtype: int64

In [6]:
df9_sample = df9[['unit_token', 'decision_event_token', 'date']].groupby('date', group_keys=False).sample(n = 100000, random_state = 2022).reset_index(drop=True)

In [7]:
df9_sample

,unit_token,decision_event_token,date
0,5F91PVV8NPESY,DS-CS_C-33SSZ673Q7RPB9_2022-09-01,2022-09-01
1,6DZHZET96BM2M,DS-CS_C-2B8AQ8J8NXZNG8_2022-09-01,2022-09-01
2,L3PR6QFFSTP5Z,DS-CS_C-38R3TVFHTN7PWD_2022-09-01,2022-09-01
3,LAFQ90TPZX2MP,DS-CS_C-XNV2XCS91HVYSN_2022-09-01,2022-09-01
4,78YS5D4RXZABW,DS-CS_C-3N9VFT38DP0W2T_2022-09-01,2022-09-01
...,...,...,...
2999995,QNMH6MZNQFFFW,DS-CS_C-TMKFBRV8BEDHHK_2022-09-30,2022-09-30
2999996,1DSQ70B01P5XR,DS-CS_C-3KX2PQECM32KPW_2022-09-30,2022-09-30
2999997,E9B4VNX08AMND,DS-CS_C-1PGZ8VY7CFV3RJ_2022-09-30,2022-09-30
2999998,L48V2NHGNQJHW,DS-CS_C-55QC02DDQX6ZKE_2022-09-30,2022-09-30


In [8]:
url9s = "gs://sq-ds-capital-prod/mrm/Eligibility_2022/TokenSample_202209.pq"
blocks.place(df9_sample, url9s)